In [4]:
import os
import glob
import numpy as np
import rasterio

# === CONFIG ===
DATA_DIR = r"E:\GOES-R Lightning Data\EAST-MEAN-FINAL"  # folder with 3 GeoTIFFs
OUTPUT_DIR = r"E:\GOES-R Lightning Data\EAST-MEAN-FINAL"

os.makedirs(OUTPUT_DIR, exist_ok=True)

# find all GeoTIFFs in input folder
files = glob.glob(os.path.join(DATA_DIR, "*.tif"))
if len(files) != 3:
    raise ValueError("Expected exactly 3 GeoTIFFs in input folder")

# Read first raster to get metadata
with rasterio.open(files[0]) as src:
    meta = src.meta.copy()
    shape = (src.height, src.width)
    sum_raster = np.zeros(shape, dtype=np.float64)

# loop over rasters to compute sum
for f in files:
    with rasterio.open(f) as src:
        data = src.read(1).astype(np.float64)
        sum_raster += data

# compute average
avg_raster = sum_raster / len(files)

# update metadata for output
meta.update(dtype=rasterio.float32, count=1)

# save function
def save_raster(arr, out_path):
    with rasterio.open(out_path, "w", **meta) as dst:
        dst.write(arr.astype(np.float32), 1)

# output paths
sum_path = os.path.join(OUTPUT_DIR, f"sum_raster.tif")
avg_path = os.path.join(OUTPUT_DIR, f"avg_raster.tif")

# save outputs
save_raster(sum_raster, sum_path)
save_raster(avg_raster, avg_path)

print("DONE")


DONE
